In [1]:
from datetime import datetime
from pathlib import Path

FECHA_HOY = datetime.now().strftime("%Y-%m-%d")
LOG_DIR = Path("D:/trading/logs/staging")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_PATH = LOG_DIR / f"nb_backtest_heuristico_{FECHA_HOY}.log"

def log_event(status, mensaje):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(f"{timestamp},{status},{mensaje}\n")
    print(f"[{timestamp}] {status}: {mensaje}")


In [2]:
from itertools import product

tp_vals = [0.04, 0.06]
sl_vals = [0.02, 0.03]

combinaciones = list(product(tp_vals, sl_vals))
log_event("OK", f"{len(combinaciones)} combinaciones definidas para backtest: {combinaciones}")


[2025-05-28 12:28:38] OK: 4 combinaciones definidas para backtest: [(0.04, 0.02), (0.04, 0.03), (0.06, 0.02), (0.06, 0.03)]


In [6]:
import subprocess

RUTA_SCRIPT = "D:/trading/scripts/core/run_backtest_heuristicoV2.py"
PYTHON_EXE = "D:/trading_env/Scripts/python.exe"

for tp, sl in combinaciones:
    log_event("START", f"Ejecutando backtest con TP={tp} SL={sl}")
    comando = [PYTHON_EXE, RUTA_SCRIPT, "--tp", str(tp), "--sl", str(sl)]

    try:
        resultado = subprocess.run(comando, capture_output=True, text=True)
        if resultado.returncode == 0:
            log_event("OK", f"Backtest TP={tp} SL={sl} ejecutado correctamente")
        else:
            log_event("ERROR", f"Backtest TP={tp} SL={sl} fallo con codigo {resultado.returncode}")
            print(resultado.stderr)
    except Exception as e:
        log_event("ERROR", f"Fallo al ejecutar backtest TP={tp} SL={sl}: {str(e)}")


[2025-05-28 12:31:56] START: Ejecutando backtest con TP=0.04 SL=0.02
[2025-05-28 12:35:26] OK: Backtest TP=0.04 SL=0.02 ejecutado correctamente
[2025-05-28 12:35:26] START: Ejecutando backtest con TP=0.04 SL=0.03
[2025-05-28 12:38:57] OK: Backtest TP=0.04 SL=0.03 ejecutado correctamente
[2025-05-28 12:38:57] START: Ejecutando backtest con TP=0.06 SL=0.02
[2025-05-28 12:42:35] OK: Backtest TP=0.06 SL=0.02 ejecutado correctamente
[2025-05-28 12:42:35] START: Ejecutando backtest con TP=0.06 SL=0.03
[2025-05-28 12:46:19] OK: Backtest TP=0.06 SL=0.03 ejecutado correctamente


In [7]:
from pathlib import Path

resultados = sorted(Path("D:/trading/reports/backtesting/").glob("resumen_metricas_tp*.csv"))
for archivo in resultados:
    print("Archivo generado:", archivo.name)


Archivo generado: resumen_metricas_tp4_sl2_2025-05-28.csv
Archivo generado: resumen_metricas_tp4_sl3_2025-05-28.csv
Archivo generado: resumen_metricas_tp6_sl2_2025-05-28.csv
Archivo generado: resumen_metricas_tp6_sl3_2025-05-28.csv
